# RNN - Model

This Notebook will be used to create and iterate over our RNN - Model to predict Memecoin Prices.

### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

### Data

In [4]:
url_train = 'https://raw.githubusercontent.com/TatevKaren/recurrent-neural-network-pricing-model/main/bitcoin_data/Bitcoin_Stock_Price_Trainset.csv'
url_test = 'https://raw.githubusercontent.com/TatevKaren/recurrent-neural-network-pricing-model/main/bitcoin_data/Bitcoin_Stock_Price_Testset.csv'

df_train = pd.read_csv(url_train,index_col=0,parse_dates=[0])
df_test = pd.read_csv(url_test,index_col=0,parse_dates=[0])

In [5]:
df_train.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-01-01,430.721008,436.246002,427.515015,434.334015,434.334015,36278900
2016-01-02,434.622009,436.062012,431.869995,433.437988,433.437988,30096600
2016-01-03,433.578003,433.743011,424.705994,430.010986,430.010986,39633800
2016-01-04,430.061005,434.516998,429.084015,433.091003,433.091003,38477500
2016-01-05,433.069000,434.182007,429.675995,431.959991,431.959991,34522600


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1823 entries, 2016-01-01 to 2020-12-31
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1823 non-null   float64
 1   High       1823 non-null   float64
 2   Low        1823 non-null   float64
 3   Close      1823 non-null   float64
 4   Adj Close  1823 non-null   float64
 5   Volume     1823 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 99.7 KB


In [17]:
training_set = df_train.iloc[:,1:2].values

In [20]:
# Feature scaling
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)
#print(training_set_scaled[0,:])

#creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []

for i in range(90,training_set_scaled.size):
    # appending the 60 previous stock prices to the list for i
    # we need to specify the rows and simply pick the first and only column
    X_train.append(training_set_scaled[i-90:i, 0])
    # appending the 60th stock price to the list for i
    y_train.append(training_set_scaled[i, 0])
# transforming pandas lists to numpy arrays required for the RNN
X_train, y_train = np.array(X_train), np.array(y_train)
#print(X_train)


## Scaling

In [22]:
X_train = np.reshape(X_train, [X_train.shape[0], X_train.shape[1], 1])

In [15]:
X_train = []
y_train = []
X_test  = []
for i in range(90,training_set_scaled.size):
    X_train.append(training_set_scaled[i-90:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
for i in range(90,inputs.size):
    X_test.append(inputs[i-90:i, 0])
X_test = np.array(X_test)

IndexError: ignored

# Modeling

In [ ]:
regressor = Sequential()

# LSTM layer 1
regressor.add(LSTM(units = 50, return_sequences=True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# LSTM layer 2,3,4
regressor.add(LSTM(units = 50, return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences=True))
regressor.add(Dropout(0.2))

# LSTM layer 5
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Fully connected layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Specifying Early Stopping Criterion
es = EarlyStopping(patience= 4, restore_best_weights=True)

#Fitting the RNN model
regressor.fit(X_train, 
              y_train, 
              epochs = 20, 
              batch_size = 32,
              callbacks= [es])

Epoch 1/20
55/55 [==============================] - 26s 292ms/step - loss: 0.0109
Epoch 2/20
55/55 [==============================] - 16s 291ms/step - loss: 0.0045
Epoch 3/20
55/55 [==============================] - 16s 299ms/step - loss: 0.0034
Epoch 4/20
55/55 [==============================] - 16s 291ms/step - loss: 0.0032
Epoch 5/20
55/55 [==============================] - 16s 293ms/step - loss: 0.0027
Epoch 6/20
55/55 [==============================] - 16s 294ms/step - loss: 0.0026
Epoch 7/20
55/55 [==============================] - 16s 287ms/step - loss: 0.0024
Epoch 8/20
55/55 [==============================] - 16s 291ms/step - loss: 0.0020
Epoch 9/20
55/55 [==============================] - 16s 294ms/step - loss: 0.0020
Epoch 10/20
55/55 [==============================] - 16s 291ms/step - loss: 0.0018
Epoch 11/20
55/55 [==============================] - 16s 294ms/step - loss: 0.0016
Epoch 12/20
55/55 [==============================] - 16s 298ms/step - loss: 0.0018
Epoch 13/20
5

In [ ]:
real_stock_price = df_test.iloc[:,1:2].values
# predicting the stock prices using X_test
# we need the original training data (vertical concatination 0, horizontal 1)
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
print(real_stock_price)

# for the consistence of the model we need to have the same scaling on the test as on the training
# for the first test day we need the previous 60 days data from train
print("First Financial day in 2021, when testing period begins",len(dataset_total)- len(dataset_test))
print("First Financial day in 2021 minus 60 days",len(dataset_total)- len(dataset_test) - 90)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 90:].values

# reshaping,normalizing the inputs
inputs = inputs.reshape(-1,1)
# no longer fitting but just transforming
inputs = sc.transform(inputs)


#storing test data as we did in teh training
X_test = []
for i in range(90,inputs.size):
    X_test.append(inputs[i-90:i, 0])
X_test = np.array(X_test)
#adding extra dimension
X_test = np.reshape(X_test, [X_test.shape[0], X_test.shape[1], 1])

#obtaining predicted values
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

#--------------------- Visualizing the RNN model results--------------------#
plt.plot(real_stock_price, color = '#ffd700', label = "Real Price January - March 2021")
plt.plot(predicted_stock_price, color = '#4782B4', label = "Predicted Price January - March 2021")
plt.title("Bitcoin Price Prediction")
plt.xlabel("Time")
plt.ylabel("Bitcoin Price")
plt.legend()
plt.show()
